# Setup

In [181]:
%matplotlib inline
from pylab import *

# importing the class stored in the same directory as this file
import detectorgeometry as geom
# including command to reload the class in case it is modified
reload(geom)

import detectorgeometry_trap as geom_trap
reload(geom_trap)

import detector_decay as decay
reload(decay)

<module 'detector_decay' from 'detector_decay.py'>

In [182]:
# example data input to class to determine effective area

theta_src = radians(40) #radians
h = 35.5 #km
R = 6356. # km
theta_view = radians(10) #degrees
N = 1000 # number of samples
E_nu = 19 #log energy in eV of incident neutrino 

# using the class
area1 = geom.Area(theta_src,h,R,theta_view, N)
area2 = geom_trap.Area(theta_src,h,R,theta_view, N)
#print area2.earth_patch()
#print area2.earth_locs()
#print degrees(area2.view_angle_det())

#print area1.degree_eff_area(), area2.degree_eff_area()

In [183]:
ret_e_dot, ret_phi_e, ret_t_e, ret_rho  = area1.event_retention()

In [184]:
# Assign to each event a tau energy and a tau decay point
decay1 = decay.tau_event(theta_src, ret_e_dot, ret_phi_e, ret_t_e, ret_rho, E_nu)
print decay1.elev
#ret_e_dot, ret_phi_e, ret_t_e, ret_tau_energy = decay1.event_retention()
print decay1.event_retention()

-50.0


ValueError: need more than 2 values to unpack

In [21]:
#np.savetxt("XXX.csv", XXX, delimiter=",")

In [154]:
TES1 = './high_statistics/4.0km_ice_midCS_stdEL/LUT_1e+15_eV.npz'
TES2 = './high_statistics/4.0km_ice_midCS_stdEL/LUT_3e+15_eV.npz'
TES3 = './high_statistics/4.0km_ice_midCS_stdEL/LUT_1e+16_eV.npz'
TES4 = './high_statistics/4.0km_ice_midCS_stdEL/LUT_3e+16_eV.npz'
TES5 = './high_statistics/4.0km_ice_midCS_stdEL/LUT_1e+17_eV.npz'
TES6 = './high_statistics/4.0km_ice_midCS_stdEL/LUT_3e+17_eV.npz'
TES7 = './high_statistics/4.0km_ice_midCS_stdEL/LUT_1e+18_eV.npz'
TES8 = './high_statistics/4.0km_ice_midCS_stdEL/LUT_3e+18_eV.npz'
TES9 = './high_statistics/4.0km_ice_midCS_stdEL/LUT_1e+19_eV.npz'
TES10 = './high_statistics/4.0km_ice_midCS_stdEL/LUT_3e+19_eV.npz'
TES11 = './high_statistics/4.0km_ice_midCS_stdEL/LUT_1e+20_eV.npz'
TES12 = './high_statistics/4.0km_ice_midCS_stdEL/LUT_3e+20_eV.npz'
TES13 = './high_statistics/4.0km_ice_midCS_stdEL/LUT_1e+21_eV.npz'

TESchoices = [TES1,TES2,TES3,TES4,TES5,TES6,TES7,TES8,TES9,TES10,TES11,TES12,TES13]       
energychoices = [15.0,15.5,16.0,16.5,17.0,17.5,18.0,18.5,19.0,19.5,20.0,20.5,21.0]

# selecting the simulation with the appropriate incident tau neutrino energy
k = energychoices.index(E_nu)
data = load(TESchoices[k])
exit_angles = data['th_exit_array']

print exit_angles

[  0.   -0.1  -0.2  -0.3  -0.4  -0.5  -0.6  -0.7  -0.8  -0.9  -1.   -1.1
  -1.2  -1.3  -1.4  -1.5  -1.6  -1.7  -1.8  -1.9  -2.   -2.1  -2.2  -2.3
  -2.4  -2.5  -2.6  -2.7  -2.8  -2.9  -3.   -3.1  -3.2  -3.3  -3.4  -3.5
  -3.6  -3.7  -3.8  -3.9  -4.   -4.1  -4.2  -4.3  -4.4  -4.5  -4.6  -4.7
  -4.8  -4.9  -5.   -6.   -7.   -8.   -9.  -10.  -11.  -12.  -13.  -14.
 -15.  -16.  -17.  -18.  -19.  -20.  -21.  -22.  -23.  -24.  -25.  -26.
 -27.  -28.  -29.  -30.  -31.  -32.  -33.  -34.  -35.  -36.  -37.  -38.
 -39.  -40.  -41.  -42.  -43.  -44.  -45.  -46.  -47.  -48.  -49.  -50.
 -51.  -52.  -53.  -54.  -55.  -56.  -57.  -58.  -59.  -60.  -61.  -62.
 -63.  -64.  -65.  -66.  -67.  -68.  -69.  -70.  -71.  -72.  -73.  -74.
 -75.  -76.  -77.  -78.  -79.  -80.  -81.  -82.  -83.  -84.  -85.  -86.
 -87.  -88.  -89. ]
